# PJM Electricity Price Prediction Analysis

This notebook provides an interactive interface for analyzing PJM electricity price data and building prediction models for daily and hourly forecasts.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Load the PJM data
print("Loading PJM data...")
data_file = 'da_hrl_lmps (1).csv'

# Load data in chunks to handle large file
chunks = []
for chunk in pd.read_csv(data_file, chunksize=100000):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

# Convert datetime columns
df['datetime_beginning_utc'] = pd.to_datetime(df['datetime_beginning_utc'])
df['datetime_beginning_ept'] = pd.to_datetime(df['datetime_beginning_ept'])

print(f"Data loaded: {len(df):,} records")
print(f"Date range: {df['datetime_beginning_utc'].min()} to {df['datetime_beginning_utc'].max()}")
print(f"Number of unique nodes: {df['pnode_id'].nunique():,}")
print(f"Number of zones: {df['zone'].nunique()}")

In [ ]:
# Display basic statistics
print("\n=== PRICE STATISTICS ===")
print(df['total_lmp_da'].describe())

# Zone analysis
print("\n=== TOP 10 ZONES BY AVERAGE PRICE ===")
zone_stats = df.groupby('zone')['total_lmp_da'].agg(['mean', 'std', 'count']).round(2)
print(zone_stats.sort_values('mean', ascending=False).head(10))

## 2. Visualize Price Patterns

In [ ]:
# Create time-based features for visualization
df['hour'] = df['datetime_beginning_utc'].dt.hour
df['day_of_week'] = df['datetime_beginning_utc'].dt.dayofweek
df['month'] = df['datetime_beginning_utc'].dt.month

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hourly pattern
hourly_avg = df.groupby('hour')['total_lmp_da'].mean()
axes[0, 0].plot(hourly_avg.index, hourly_avg.values, marker='o')
axes[0, 0].set_title('Average Price by Hour of Day')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Price ($/MWh)')
axes[0, 0].grid(True, alpha=0.3)

# Daily pattern
daily_avg = df.groupby('day_of_week')['total_lmp_da'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[0, 1].bar(range(7), daily_avg.values)
axes[0, 1].set_title('Average Price by Day of Week')
axes[0, 1].set_xlabel('Day of Week')
axes[0, 1].set_ylabel('Price ($/MWh)')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(days)
axes[0, 1].grid(True, alpha=0.3)

# Monthly pattern
monthly_avg = df.groupby('month')['total_lmp_da'].mean()
axes[1, 0].plot(monthly_avg.index, monthly_avg.values, marker='o')
axes[1, 0].set_title('Average Price by Month')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Price ($/MWh)')
axes[1, 0].grid(True, alpha=0.3)

# Price distribution
axes[1, 1].hist(df['total_lmp_da'], bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Price Distribution')
axes[1, 1].set_xlabel('Price ($/MWh)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Zone-Specific Analysis

In [ ]:
# Analyze specific zones
print("Available zones:")
available_zones = df['zone'].dropna().unique()
for i, zone in enumerate(available_zones[:10]):  # Show first 10
    print(f"{i+1}. {zone}")

# Select a zone for detailed analysis (you can change this)
target_zone = available_zones[0] if len(available_zones) > 0 else None
print(f"\nAnalyzing zone: {target_zone}")

if target_zone:
    zone_data = df[df['zone'] == target_zone].copy()
    
    # Time series plot for the selected zone
    plt.figure(figsize=(15, 6))
    plt.plot(zone_data['datetime_beginning_utc'], zone_data['total_lmp_da'], alpha=0.7)
    plt.title(f'Price Time Series for {target_zone}')
    plt.xlabel('Date')
    plt.ylabel('Price ($/MWh)')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"\nStatistics for {target_zone}:")
    print(zone_data['total_lmp_da'].describe())

## 4. Price Components Analysis

In [ ]:
# Analyze price components
component_cols = ['system_energy_price_da', 'congestion_price_da', 'marginal_loss_price_da']

# Check if components exist
available_components = [col for col in component_cols if col in df.columns]

if available_components:
    print("=== PRICE COMPONENTS ANALYSIS ===")
    
    # Correlation matrix
    corr_data = df[['total_lmp_da'] + available_components].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.3f')
    plt.title('Price Components Correlation Matrix')
    plt.show()
    
    # Component contributions over time
    if target_zone:
        zone_data = df[df['zone'] == target_zone].copy()
        
        # Sample data for visualization (take every 24th hour to reduce density)
        sample_data = zone_data.iloc[::24].copy()
        
        plt.figure(figsize=(15, 8))
        plt.plot(sample_data['datetime_beginning_utc'], sample_data['total_lmp_da'], 
                label='Total LMP', linewidth=2)
        
        if 'system_energy_price_da' in available_components:
            plt.plot(sample_data['datetime_beginning_utc'], sample_data['system_energy_price_da'], 
                    label='Energy Component', alpha=0.7)
        if 'congestion_price_da' in available_components:
            plt.plot(sample_data['datetime_beginning_utc'], sample_data['congestion_price_da'], 
                    label='Congestion Component', alpha=0.7)
        if 'marginal_loss_price_da' in available_components:
            plt.plot(sample_data['datetime_beginning_utc'], sample_data['marginal_loss_price_da'], 
                    label='Loss Component', alpha=0.7)
        
        plt.title(f'Price Components Over Time for {target_zone}')
        plt.xlabel('Date')
        plt.ylabel('Price ($/MWh)')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("Price component data not available in the dataset")

## 5. Run the Prediction Model

In [ ]:
# Import and run the prediction model
from pjm_price_prediction import PJMPricePredictor

# Initialize predictor
predictor = PJMPricePredictor('da_hrl_lmps (1).csv')

# Load data (reuse already loaded data to save time)
predictor.data = df

# Create features
print("\nCreating features for modeling...")
feature_df = predictor.create_features(target_zone=target_zone)

# Prepare train/test split
train_df, test_df = predictor.prepare_train_test(feature_df, test_days=7)

# Train models
results, best_model = predictor.train_models(train_df, test_df)

print(f"\nBest performing model: {best_model}")
print(f"Best MAE: ${results[best_model]['mae']:.2f}")

## 6. Model Results Visualization

In [ ]:
# Create detailed visualizations
predictor.create_visualizations(test_df, results)

# Additional analysis: Feature importance
if 'Random Forest' in results:
    rf_model = results['Random Forest']['model']
    feature_cols = predictor.scalers['feature_cols']
    
    # Get feature importance
    importance = rf_model.feature_importances_
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Plot top 15 features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Most Important Features (Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))

## 7. Summary and Recommendations

In [ ]:
# Print summary
print("=== PJM PRICE PREDICTION ANALYSIS SUMMARY ===")
print(f"Dataset: {len(df):,} records")
print(f"Date range: {df['datetime_beginning_utc'].min()} to {df['datetime_beginning_utc'].max()}")
print(f"Analyzed zone: {target_zone}")
print(f"Best model: {best_model}")
print(f"Best MAE: ${results[best_model]['mae']:.2f}")
print(f"Best RMSE: ${results[best_model]['rmse']:.2f}")
print(f"Best R²: {results[best_model]['r2']:.3f}")

print("\n=== RECOMMENDATIONS FOR IMPROVEMENT ===")
print("1. Add external data sources:")
print("   - Weather forecasts (temperature, wind speed, solar irradiance)")
print("   - Load forecasts")
print("   - Fuel prices (natural gas, coal)")
print("   - Generation outage schedules")
print("\n2. Advanced modeling techniques:")
print("   - LSTM/GRU neural networks for sequence modeling")
print("   - Ensemble methods combining multiple models")
print("   - Transfer learning from similar markets")
print("\n3. Feature engineering:")
print("   - Weather-price interaction terms")
print("   - Holiday and special event indicators")
print("   - Market constraint indicators")